# Проверка диапазона

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn import linear_model

template = 'ggplot2'

### вводим данные, снятые на базе БелГИМ

In [22]:
data = pd.DataFrame(columns=['n_sorce', 'activ', '1/min*cm2', 'Bk/cm2', 's-1', 'n'])

data.loc[len(data)] = [11577, 1339.6, [3550, 3580, 3490], [153, 153, 155], [220, 219, 220], [200, 175, 192, 220, 180, 185, 218, 212, 206, 175, 176, 198]]
data.loc[len(data)] = [1389, 227, [610, 614, 613], [26.5, 26.3, 26.4], [38.9, 37.7, 37.4], [28, 50, 38, 32, 29, 40, 33, 26, 36, 26, 33, 26, 46, 28, 39, 43, 42, 33, 41, 22]]
data.loc[len(data)] = [11382, 866.9, [2520, 2500, 2510], [110, 112, 112], [160, 156, 157], [161, 124, 149, 115, 152, 170, 141, 150, 151, 114, 118, 152]]
data.loc[len(data)] = [1386, 24.2, [64.8, 68.4, 74.5], [3.06, 3.26, 3.13], [4.37, 4.46], [3, 4, 7, 4, 1, 4, 6, 4, 5, 2, 6, 4, 2, 9, 5, 3, 4, 4, 9, 3]]
data.loc[len(data)] = [1392, 2210, [5830, 5670, 5660], [259, 260, 266], [379, 378, 380], [297, 326, 358, 333, 336, 320, 313, 340, 353, 347, 317]]

data['n_mean'] = [np.mean(i) for i in data['n']]
data['1/min*cm2'] = [np.mean(i) for i in data['1/min*cm2']]
data['Bk/cm2'] = [np.mean(i) for i in data['Bk/cm2']]
data['s-1'] = [np.mean(i) for i in data['s-1']]

data.sort_values(by='activ', inplace=True)
data.head()

,n_sorce,activ,1/min*cm2,Bk/cm2,s-1,n,n_mean
3,1386,24.2,69.233333,3.150000,4.415000,"[3, 4, 7, 4, 1, 4, 6, 4, 5, 2, 6, 4, 2, 9, 5, ...",4.450000
1,1389,227.0,612.333333,26.400000,38.000000,"[28, 50, 38, 32, 29, 40, 33, 26, 36, 26, 33, 2...",34.550000
2,11382,866.9,2510.000000,111.333333,157.666667,"[161, 124, 149, 115, 152, 170, 141, 150, 151, ...",141.416667
0,11577,1339.6,3540.000000,153.666667,219.666667,"[200, 175, 192, 220, 180, 185, 218, 212, 206, ...",194.750000
4,1392,2210.0,5720.000000,261.666667,379.000000,"[297, 326, 358, 333, 336, 320, 313, 340, 353, ...",330.909091


### Отображаем полученные данные для предварительной оценки

In [24]:
fig = px.line(data, y=['1/min*cm2', 'Bk/cm2', 's-1', 'n_mean'], x='activ' , markers='*', log_y=True, log_x=True, template=template)
fig.update_layout(
                  title="Показания прибора",
                  xaxis_title="Энергия, МэВ (логарифмическая шкала)",
                  yaxis_title="отсчеты в соответствующих единицах (логарифмическая шкала)",
)
fig.show()

### Оценим разброс моментальной скорости счета.

In [4]:
fig = px.line(data, y = 'n_mean', x='activ' , markers='*')
fig
for i, row in data.iterrows():
  fig.add_trace(go.Box(y=row[5], 
                       name=row[1]
                       )
  )
fig.update_layout(legend_orientation="h",
                  title="Показания прибора",
                  xaxis_title="Энергия, МэВ",
                  yaxis_title="скорость счета",
)
fig.show()

### Приведем все показания прибора к одним единицам, чтобы увеличить колличество данных для последующей аппроксимации.

In [33]:
df = pd.DataFrame(columns=['activ', 's-1'])
for i, row in data.iterrows():
  df.loc[i] = [row[1], (row[2]/15.9 + row[3]/0.7 + row[4]) / 3]
df

,activ,s-1
3,24.2,4.423099
1,227.0,38.075272
2,866.9,158.191974
0,1339.6,220.610662
4,2210.0,370.852650


### Строим линейную аппроксимацию по точкам в рабочем диапазоне (24.2, 227)Бк*см2.
### Рассчитываем отклонения реальных показаний от расчетной линии

In [41]:
reg = linear_model.LinearRegression()
x = df['s-1'].values[:2]
y = df['activ'].values[:2]
reg.fit(x.reshape(-1, 1), y.reshape(-1, 1))
x0 = df['s-1'].values
y_predicted = reg.predict(x0.reshape(-1, 1))
print(f'approcsimation parametrs  : w1={reg.coef_}, w0={reg.intercept_}')

data_1 = pd.DataFrame()
data_1['x'] = df['s-1']
data_1['y'] = df['activ']
data_1['y_predicted'] = y_predicted

delta = np.round((data_1['y'] - data_1['y_predicted']) * 100 / data_1['y'], 1).astype('str') + ' %'
delta.index = data_1['y']

delta

approcsimation parametrs  : w1=[[6.02635679]], w0=[-2.45517407]


y
24.2      -0.0 %
227.0      0.0 %
866.9     -9.7 %
1339.6     0.9 %
2210.0    -1.0 %
dtype: object

In [42]:
fig_1 = px.line(data_1,  x='x', y=['y', 'y_predicted'], markers='*')
fig_1.update_layout(legend_orientation="h",
                  title="Показания прибора",
                  xaxis_title="Энергия, МэВ",
                  yaxis_title="скорость счета",
)
fig_1.show()